# Evaluation of Cross-modal Query Suggestion methods on CroQS benchmark

In [1]:
device_str = "cuda:0"
device_str_llm = device_str

import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from lib.CroQS import CroQS

croQS_json_path = "./2024-10-07_15-35-43.json"
croQS = CroQS(croQS_json_path)
device = "cuda:0"
hdf5_file_path = os.getenv("HDF5_INDEX_FILE_PATH")
croQS.evaluation_init(hdf5_file_path, device)

c:\Python310\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


No self.queries dict object loaded from disk, creating new empty dict


In [3]:
len(croQS.list_queries())

50

In [4]:
from lib.methodsLoader import get_qe_methods_dict, add_method_to_dict

qe_methods_names = [
    'initial-query',
    #'clipcap-on-centroids', 
    #'decap-on-centroids', 
    ##'clipcap-query_set-of-embeddings', 
    #'decap-on-representatives', 
    #'clipcap-on-representatives',
    #'clipcap-prompting-on-centroids',
    #'clipcap-prompting-on-representatives',
    ]

In [5]:
def load_qe_dict(ir_system, load_group_cap : bool = False):
    from lib.methods.decap.Im2TxtProjection import Im2TxtProjector, ProjectionType
    from lib.methods.decap.DecapQueryExpansion import DeCapQueryExpansion
    from lib.methods.decap.decap import get_decap_model
    from lib.methods.groupcap.model import GroupCapLLM, LLMBasePrompts, ImageCaptioningMethods, LLMTypes

    decap_qe = DeCapQueryExpansion.load_object(device_str, 'coco')
    assert isinstance(decap_qe, DeCapQueryExpansion)
    decap_qe.im2txt.ir_system = ir_system
    qe_dict = get_qe_methods_dict(methods_keys=qe_methods_names, decap_qe=decap_qe, device=device_str)

    GC_TYPE = "gemma2-2b" # "llama3-8b", "mistral7b"

    if load_group_cap:

        if GC_TYPE == "gemma2b":
            llm_type = LLMTypes.Gemma2b_IT
        elif GC_TYPE == "gemma2-2b":
            llm_type = LLMTypes.Gemma2_2b
        elif GC_TYPE == "mistral7b":
            llm_type = LLMTypes.Mistral7b_IT
        elif GC_TYPE == "llama3-8b":
            llm_type = LLMTypes.LLama3_8b_IT
        else:
            print(f"Using default LLM - LLama3-8b")
            GC_TYPE = "llama3-8b"
            llm_type = LLMTypes.LLama3_8b_IT

        group_cap_obj = GroupCapLLM(ImageCaptioningMethods.decap, decap_qe.get_generated_captions, llm_type, device_map=device_str_llm, image_captioning_method_device=device_str)
        
        prompts_dict = GroupCapLLM.get_prompts_dict()

        group_cap_obj.qe_prompt = prompts_dict['query-expansion']['qe-few-shot-1']
        qe_dict = add_method_to_dict(qe_dict, f'groupcap-{GC_TYPE}-qe-few-shot-1', group_cap_obj.generate_expanded_query, wants_query=True, method_group_name="img-embeddings-whole-cluster")

    return qe_dict

qe_dict = load_qe_dict(croQS.ir_system, True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
croQS.get_query_suggestions("dog running", "0")

{'initial-query': 'dog running',
 'human': 'dog running catching a frisbee',
 'clipcap-on-centroids': 'A dog is jumping to catch a frisbee.',
 'decap-on-centroids': 'a dog running to catch a frisbee in its mouth . ',
 'groupcap-gemma2b-qe-few-shot-1': 'The expanded query "dog running" is not included in the initial query.',
 'groupcap-gemma2-2b-qe-few-shot-1': '\n'}

In [8]:
croQS.load_query_suggestions("dog running", "0", qe_dict, force_update=True)

The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.
c:\Python310\lib\site-packages\bitsandbytes\nn\modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Computing scores for cluster '0' of query = 'dog running'


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[ndcg] [W] division by 0 [W]


({'initial-query': 'dog running',
  'human': 'dog running catching a frisbee',
  'clipcap-on-centroids': 'A dog is jumping to catch a frisbee.',
  'decap-on-centroids': 'a dog running to catch a frisbee in its mouth . ',
  'groupcap-gemma2b-qe-few-shot-1': 'The expanded query "dog running" is not included in the initial query.',
  'groupcap-gemma2-2b-qe-few-shot-1': 'Initial Query: cat drinking\r\nCaptions:\r\n\t• A small tabby cat is on top of a toilet\r\n\t• A cat batting at the water in a toilet\r\n\t• A small kitten sits on the rim of a toilet seat\r\n\t• A cat stands on a toilet seat and peers into the bowl\r\n\t• A cat that is sitting on a toilet seat\r\nExpanded query:\r\ncat drinking from the toilet\r\n\r\n\r\nInitial query: horse sleeping\r\nCaptions:\r\n\t• Several horses standing and lying down in a grassy field.\r\n\t• A horse and a donkey stand in a fenced field.\r\n\t• Two horses graze in a grassy pasture lined with pine trees.\r\n\t• A brown horse standing in a field nex

In [9]:
ret_df = croQS.get_query_suggestions_dataframe_all_queries(qe_dict)

ret_df

Loading already existing qs for the method 'initial-query'oids's'Computing scores for cluster '2' of query = 'dog running'
Loading already existing qs for the method 'initial-query'Computing scores for cluster '0' of query = 'person reading'
Loading already existing qs for the method 'initial-query'Computing scores for cluster '1' of query = 'person reading'
Loading already existing qs for the method 'initial-query'Computing scores for cluster '2' of query = 'person reading'
Loading already existing qs for the method 'initial-query'Computing scores for cluster '3' of query = 'person reading'
Loading already existing qs for the method 'initial-query'Computing scores for cluster '0' of query = 'laptop near'
Loading already existing qs for the method 'initial-query'Computing scores for cluster '1' of query = 'laptop near'
Loading already existing qs for the method 'initial-query'Computing scores for cluster '2' of query = 'laptop near'
Loading already existing qs for the method 'initial-q

,query,cluster,method,suggested-query,jaccard-q,clip-similarity-cluster,clip-similarity-q,NDCG@10,MAP,Recall-Open-Set-@100,Recall-Closed-Set
0,dog running,0,initial-query,dog running,1.00,0.293,1.000,0.408536,0.497725,0.548387,0.720430
1,dog running,0,human,dog running catching a frisbee,0.50,0.319,0.830,0.864315,0.741513,0.580645,0.892473
2,dog running,0,clipcap-on-centroids,A dog is jumping to catch a frisbee.,0.20,0.328,0.813,0.930569,0.664155,0.537634,0.913978
3,dog running,0,decap-on-centroids,a dog running to catch a frisbee in its mouth .,0.40,0.329,0.808,0.933746,0.767587,0.612903,0.913978
4,dog running,2,initial-query,dog running,1.00,0.294,1.000,0.215018,0.214729,0.500000,0.294118
...,...,...,...,...,...,...,...,...,...,...,...
1175,bridge,5,decap-on-centroids,a train traveling over a bridge over a river .,0.25,0.310,0.805,0.643121,0.546846,0.629630,0.481481
1176,bridge,6,initial-query,bridge,1.00,0.270,1.000,0.138862,0.213246,0.606061,0.181818
1177,bridge,6,human,bridge over a river near forest,0.25,0.266,0.897,0.599694,0.456155,0.666667,0.363636
1178,bridge,6,clipcap-on-centroids,A train traveling over a bridge over a river.,0.25,0.292,0.805,0.066254,0.255850,0.515152,0.363636


### Micro averaged results

In [14]:
grouped_df = ret_df.groupby(by='method').mean(numeric_only=True).reset_index()#.sort_values(by=sort_by_column, ascending=False)

grouped_df

,method,jaccard-q,clip-similarity-cluster,clip-similarity-q,NDCG@10,MAP,Recall-Open-Set-@100,Recall-Closed-Set
0,clipcap-on-centroids,0.134471,0.292264,0.722292,0.284567,0.257788,0.401399,0.515541
1,decap-on-centroids,0.155285,0.303275,0.767603,0.299149,0.268678,0.396892,0.495582
2,human,0.437193,0.299312,0.870451,0.497983,0.430224,0.626201,0.571772
3,initial-query,1.000000,0.280369,1.000000,0.162076,0.194837,0.515198,0.170162


### Macro averaged results

In [16]:
import pandas as pd

tmp_df = None

for q in ret_df['query'].unique().tolist():
    df_q = ret_df[ret_df['query'] == q]
    grouped_df_q = df_q[ grouped_df.columns.to_list() + ['query', 'cluster', 'suggested-query'] ].groupby(by='method').mean(numeric_only=True).reset_index()#.sort_values(by=sort_by_column, ascending=False)
    
    if tmp_df is None:
        tmp_df = grouped_df_q
    else:
        tmp_df = pd.concat([tmp_df, grouped_df_q], ignore_index=True)

macro_averaged_df = tmp_df.groupby(by='method').mean(numeric_only=True).reset_index()#.sort_values(by=sort_by_column, ascending=False)
macro_averaged_df

,method,jaccard-q,clip-similarity-cluster,clip-similarity-q,NDCG@10,MAP,Recall-Open-Set-@100,Recall-Closed-Set
0,clipcap-on-centroids,0.137081,0.291701,0.722223,0.296909,0.267135,0.397572,0.545929
1,decap-on-centroids,0.155587,0.301944,0.766995,0.311279,0.279430,0.391299,0.522512
2,human,0.437972,0.298296,0.870624,0.518537,0.445149,0.620265,0.591542
3,initial-query,1.000000,0.279894,1.000000,0.177377,0.210269,0.518257,0.190265


In [11]:
qq = 'person at the phone'
cc = '3'

croQS._list_clusters_labels(qq)

['0', '1', '2', '3']

In [12]:
croQS.suggested_queries[qq][cc]

{'suggested-queries': {'initial-query': {'query': 'person at the phone',
   'scores': {'jaccard-q': 1.0,
    'clip-similarity-cluster': 0.294,
    'clip-similarity-q': 1.0,
    'NDCG@10': 0.23365080824310536,
    'MAP': 0.2868582289545055,
    'Recall-Open-Set-@100': 0.5128205128205128,
    'Recall-Closed-Set': 0.28205128205128205}},
  'human': {'query': 'woman talking on mobile phone',
   'scores': {'jaccard-q': 0.2,
    'clip-similarity-cluster': 0.317,
    'clip-similarity-q': 0.877,
    'NDCG@10': 0.39639187290150935,
    'MAP': 0.3438731991303363,
    'Recall-Open-Set-@100': 0.5128205128205128,
    'Recall-Closed-Set': 0.5897435897435898}},
  'clipcap-on-centroids': {'query': 'A woman with a ring on her hand talking on a cell phone.',
   'scores': {'jaccard-q': 0.143,
    'clip-similarity-cluster': 0.321,
    'clip-similarity-q': 0.785,
    'NDCG@10': 0.15840915256850252,
    'MAP': 0.22080860477453584,
    'Recall-Open-Set-@100': 0.5897435897435898,
    'Recall-Closed-Set': 0.512